In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
# import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [3]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv("../input/test.csv")
train_df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [4]:
train_df['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [5]:
from nltk.corpus import stopwords

In [6]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [7]:
train_text = train_df['question_text']
test_text = test_df['question_text']
train_target = train_df['target']
all_text = train_text.append(test_text)

In [9]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(all_text)

count_vectorizer = CountVectorizer(max_df=0.8,ngram_range=(1, 1))# 未考虑停用词对分词效果的影响，以及对分词结果的统计量作为新的特征，需要把
count_vectorizer.fit(all_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [10]:
kfold = KFold(n_splits = 5, shuffle = True, random_state = 43)
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
test_preds = 0
oof_preds = np.zeros([train_df.shape[0],])

train_text_features_cv = count_vectorizer.transform(train_text)
test_text_features_cv = count_vectorizer.transform(test_text)

# train_text_features_tf = tfidf_vectorizer.transform(train_text)
# test_text_features_tf = tfidf_vectorizer.transform(test_text)


In [1]:
from sklearn import svm
test_preds = 0
# for i, (train_idx,valid_idx) in enumerate(kfold.split(train_df)):
for i ,(train_idx,valid_idx) in enumerate(skf.split(train_df, train_df['target'])):
    x_train, x_valid = train_text_features_cv[train_idx,:], train_text_features_cv[valid_idx,:]
    y_train, y_valid = train_target[train_idx], train_target[valid_idx]
    classifier = LogisticRegression(C=3,solver='lbfgs',max_iter=200,penalty='l2')
    # liblinear can not use more than one processors
#     classifier = RandomForestClassifier(n_estimators=100, max_depth=5,n_jobs=-1)
    
#     classifier = svm.SVC()
    print('fitting.......')
    classifier.fit(x_train,y_train)
    print('predicting......')
    print('\n')
    oof_preds[valid_idx] = classifier.predict_proba(x_valid)[:,1]
    test_preds += 0.2*classifier.predict_proba(test_text_features_cv)[:,1]

NameError: name 'skf' is not defined

In [ ]:
pred_train = (oof_preds > .15).astype(np.int)
print(pred_train)
f1_score(train_target, pred_train)
# rf_df= pd.DataFrame()
# np.array(oof_preds).max()

In [ ]:
submission1 = pd.DataFrame.from_dict({'qid': test_df['qid']})
submission1['prediction'] = (test_preds>0.14).astype(np.int)
submission1.to_csv('submission.csv', index=False)
submission1['prediction'] = (test_preds>0.14)